In [1]:
import cv2
import os

# Flow
<ol>
    <li>Register face if not registered  </li>
    <li>Continue face recognition as requested by the server (Not actually implemented in this example)  </li>
    <li>Capture and process image</li>
 </ol>

In [2]:
cv2.__version__

'4.7.0'

In [9]:
# https://076923.github.io/posts/Python-opencv-2/
# https://scribblinganything.tistory.com/472

def capture():
    capture= cv2.VideoCapture(0)    # If the webcam is not installed, the CSI Cam becomes the #0 camera
    capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    while True:            # wait 33ms for key input, if key input detected, break Loop
        capture_result, frame = capture.read()        # grab() and retrieve() in one call
        cv2.imshow("VideoFrame", frame)
        if cv2.waitKey(1) == ord('c'):
            break

    capture.release()
    cv2.destroyAllWindows()
    
    return capture_result, frame

In [4]:
# https://ai0.kr/m/41

def track_faces(capture, classifier, image_scale, max_faces):
    capture_copy = capture.copy()
    grayscale_capture = cv2.cvtColor(capture_copy, cv2.COLOR_BGR2GRAY)    # Apply Grayscale
    detected_faces = classifier.detectMultiScale(grayscale_capture, image_scale, max_faces)
    face_grids = []
    
#     LINE_THICKNESS = 2
#     LINE_COLOR = (255,0,0)
    for(x, y, w, h) in detected_faces:
        # cv2.rectangle(capture_copy,(x, y), (x + w, y + h), LINE_COLOR, LINE_THICKNESS)
        face_grids.append(((x, y), (x + w, y + h)))
#     face_grids = sorted(face_grids, key = lambda x:x[1], reverse = True)
    
    return face_grids

In [7]:
def main():
    LINE_THICKNESS = 2
    LINE_COLOR = (255,0,0)
    classifier = cv2.CascadeClassifier("data/haarcascade_frontalface_default.xml")
    # Set Camera width and height (640 * 480)
    camera_width = 640
    camera_height = 480
    
    while True:
        option = int(input("1: Register\n2: Detect\nselect option: "))
       
        if option == 1:
            user_name = input("Input User Name: ").strip()
            capture_result, frame = capture()
            
            if capture_result == 1: 
                if os.path.isfile(f"data/sub_data/{user_name}.png"):
                    os.remove(f"data/sub_data/{user_name}.png")
                cv2.imwrite(f"data/sub_data/{user_name}.png", frame)
                    
                img_without_grid = cv2.imread(f"data/sub_data/{user_name}.png")
                face_grids = track_faces(frame, classifier, 1.1, 2)
                
                for grid in face_grids:
                    cv2.rectangle(img_without_grid, *grid, LINE_COLOR, LINE_THICKNESS)
                    
                cv2.imshow("Result", img_without_grid)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
        elif(option == 2):
            print("Detect From Image")
            break
            
        else:
            break

    cv2.destroyAllWindows()


In [10]:
main()

1: Register
2: Detect
select option: 1
Input User Name: lee
1: Register
2: Detect
select option: 2
Detect From Image
